https://arxiv.org/pdf/2110.01368.pdf

https://arxiv.org/pdf/1911.03380.pdf

https://deliverypdf.ssrn.com/delivery.php?ID=773124118097097119026017080102079065057081049043000029071005117069008065110098087085056017039010001111005112084088095103006007043087058092007068069090091015080092022071021048008071122094090110120127079071113126114121118104123004007070096119111104095125&EXT=pdf&INDEX=TRUE

Let $\Sigma_t$ denote portfolio of the instruments under our managment
$$\Sigma_t = n_i(t) \cdot I_i(t),$$
where $n_i(t)$ -- position in the instrument $i$, $I_i(t)$ -- its present value

$$\dot\Sigma(t) = \dot n_i I_i + n_i \dot I_i$$

So we must implement Instruments' dynamics and portolio strategy to control $n_i$ 

In [1]:
import numpy as np
import pandas as pd
from LPtools.utils import Adapter
from LPtools.clients import Client
from LPtools.instruments import UniPool, Bond, Portfolio, Position, Cash, Holder

In [2]:
zcurve = pd.read_csv('zcyc_rates.csv', index_col=0, parse_dates=True, infer_datetime_format=True, na_filter=False)
#zcurve = zcurve.reindex(pd.date_range(zcurve.index.min(), zcurve.index.max()), method='pad').to_csv('zcyc_rates.csv')

adapter = Adapter(pool_id="0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640",
                  symbol="ETHUSDT",
                  zcurve=zcurve)

response = adapter(20)
#response.token1Price = 1. # for some reason retrieved USDC value is equal to ~0.0003,  $1 expected

In [3]:
# however for some reason retrieved USDC value is equal to ~0.0003 instead of $1 expected
# it leads to wrong numbers from graphql for the other columns, binance data is ok.

# it is also interesting how uniswap price differs from binance market price

# or I don't understand sqrtPrice and other fields values correctly

response['difference'] = 1 / response.token1Price
response.relative_price = response.relative_price / response.difference
response.liquidity =  response.liquidity / response.difference
response.token1Price = 1

#after correction there are still errors left, so quering graphql and
#         making sense out of units of measure is still required

In [4]:
runner = response.itertuples()
print(response.shape)
response.head()

(20, 15)


,fundingRate,Open,Volume,date,feesUSD,liquidity,sqrtPrice,token0Price,token1Price,tvlUSD,volumeUSD,relative_price,mark,risk_free_rate,difference
2021-11-11,0.000430,4632.42,1146003.874,1.636589e+09,326104.963296,9.012761e+15,1.152457e+33,4726.173683,1,2.466958e+08,6.522099e+08,4726.173683,4632.22000000,7.80,4726.173683
2021-11-12,0.000300,4721.19,1671025.399,1.636675e+09,418567.063516,7.717919e+15,1.159227e+33,4671.134620,1,2.452712e+08,8.371341e+08,4671.134620,4721.19000000,7.68,4671.134620
2021-11-13,0.000300,4665.99,812743.805,1.636762e+09,190052.962792,7.620267e+15,1.162098e+33,4648.077390,1,2.432830e+08,3.801059e+08,4648.077390,4666,7.68,4648.077390
2021-11-14,0.000300,4644.42,947148.676,1.636848e+09,244430.697314,8.693900e+15,1.164521e+33,4628.759566,1,2.454891e+08,4.888614e+08,4628.759566,4644.51360229,7.68,4628.759566
2021-11-15,0.000575,4629.75,1351110.691,1.636934e+09,285744.523559,7.705942e+15,1.173031e+33,4561.840386,1,2.439104e+08,5.714890e+08,4561.840386,4629.44000000,7.66,4561.840386


In [5]:
init_state = next(runner)

In [6]:
cash = Cash(0)

pool = Position(tag='pool', instrument=UniPool(init_state, 2000, 5000, .005))
bond = Position(tag='bond', instrument=Bond(init_state))

cash += pool.rebalance(1000) + bond.rebalance(-1000) # create positions, suffer transaction costs

positions = [pool, bond]
balancer = Holder(positions)
portfolio = Portfolio(balancer, cash=cash, positions=positions)

In [7]:
portfolio.rollout(runner)

In [8]:
portfolio.summary

{'pool_last_val': [169.86761527331817], 'bond_last_val': [-1004.4401707129609], 'total_value': [-854.5725554374943]}


,pool,bond,payments,transaction_costs
0,0.171824,1.000217,7.476699e-11,0
1,0.170596,1.000430,3.445307e-11,0
2,0.169565,1.000643,3.890395e-11,0
3,0.165976,1.000857,5.160384e-11,0
4,0.147209,1.001070,4.313189e-10,0
5,0.150947,1.001289,9.054155e-11,0
6,0.134740,1.001508,1.521848e-10,0
7,0.151530,1.001732,8.967561e-11,0
8,0.158044,1.001957,6.072473e-11,0
9,0.150063,1.002182,6.460337e-11,0
